In [1]:
from acevedo_clss_and_fcns import * 
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda:0'
#device = torch.device(device)
print(f"{device = }")
from torch_geometric.nn import GNNExplainer

    

model = torch.load(
"./results/trained_pytorch_models/Masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_09min_best_ValAcc_0.9121086382497171_epoch_46.pt")
#loader_path = "./results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt"
loader_path = "./results/dataloaders/loader_Concen_plus_Fluxes.pt"
loader = torch.load(loader_path)
loader_validation = loader.get_validation_loader()
loader_train = loader.get_train_loader()
############################
##############################
explainer  = GNNExplainer(model, epochs=1000, num_hops = None, lr=0.01).to(device, non_blocking=True)
model.to(device, non_blocking=True)
a_batch = next(iter(loader_validation)).to(device, non_blocking=True)

train_labels = []
for graph in tqdm.tqdm(loader_train.dataset):
    train_labels.append(graph.y.cpu().item())
train_labels.__len__()


/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = 'cuda:0'


100%|██████████| 4921/4921 [00:00<00:00, 741811.74it/s]


4921

In [ ]:
model.eval()
gc.collect()
torch.cuda.empty_cache() 
edge_num           = loader_train.dataset[0].edge_index.shape[1]
all_graphs_weights = []#torch.empty(edge_num,1).to(device)

for graph in loader_train.dataset:
    _, edge_weights =  explainer.explain_graph(graph.x.to(device, non_blocking=True), graph.edge_index.to(device, non_blocking=True))
    #all_graphs_weights = torch.cat((all_graphs_weights, edge_weights.reshape(edge_num,1).to(device)), dim=1)
    all_graphs_weights.append(edge_weights.reshape(edge_num,1))
    